In [3]:
from app import app, db, bcrypt
from models import User, Stock, Portfolio
from flask import jsonify, request
import yfinance as yf
import pandas as pd
import sqlite3

In [4]:
multipleStocks = yf.Tickers('HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS AUROPHARMA.NS BBTC.NS BALKRISIND.NS GARFIBRES.NS COLPAL.NS NESTLEIND.NS MARUTI.NS BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS BAJFINANCE.NS')

In [5]:
ms_stockprices = multipleStocks.history(period="max")

[*********************100%***********************]  18 of 18 completed


In [6]:
ms_stockprices

Price              Close                                            \
Ticker     ABBOTINDIA.NS ASIANPAINT.NS AUROPHARMA.NS BAJFINANCE.NS   
Date                                                                 
1996-01-01           NaN           NaN           NaN           NaN   
1996-01-02           NaN           NaN           NaN           NaN   
1996-01-03           NaN           NaN           NaN           NaN   
1996-01-04           NaN           NaN           NaN           NaN   
1996-01-05           NaN           NaN           NaN           NaN   
...                  ...           ...           ...           ...   
2025-01-13  28555.800781   2252.649902   1161.400024   7150.950195   
2025-01-14  28235.550781   2239.949951   1174.500000   7335.000000   
2025-01-15  27745.699219   2229.699951   1147.550049   7177.399902   
2025-01-16  27298.099609   2215.600098   1156.300049   7259.750000   
2025-01-17  27929.699219   2261.000000   1178.349976   7182.100098   

Price                                                                          \
Ticker     BALKRISIND.NS      BBTC.NS   BOSCHLTD.NS BRITANNIA.NS    COLPAL.NS   
Date                                                                            
1996-01-01           NaN          NaN           NaN          NaN          NaN   
1996-01-02           NaN          NaN           NaN     7.473685          NaN   
1996-01-03           NaN          NaN           NaN     7.379674          NaN   
1996-01-04           NaN          NaN           NaN     7.379674          NaN   
1996-01-05           NaN          NaN           NaN     7.384377          NaN   
...                  ...          ...           ...          ...          ...   
2025-01-13   2595.750000  1947.900024  30979.750000  4927.049805  2752.050049   
2025-01-14   2705.350098  1977.300049  31621.699219  4862.399902  2713.300049   
2025-01-15   2680.300049  1991.599976  31463.949219  4877.000000  2664.750000   
2025-01-16   2725.300049  2000.900024  31354.349609  4844.549805  2652.550049   
2025-01-17   2705.850098  2003.300049  31399.800781  4854.399902  2683.649902   

Price                    ...    Volume                                       \
Ticker      DIVISLAB.NS  ... COLPAL.NS DIVISLAB.NS GARFIBRES.NS HDFCBANK.NS   
Date                     ...                                                  
1996-01-01          NaN  ...       NaN         NaN          NaN      350000   
1996-01-02          NaN  ...       NaN         NaN          NaN      412000   
1996-01-03          NaN  ...       NaN         NaN          NaN      284000   
1996-01-04          NaN  ...       NaN         NaN          NaN      282000   
1996-01-05          NaN  ...       NaN         NaN          NaN      189000   
...                 ...  ...       ...         ...          ...         ...   
2025-01-13  5732.500000  ...  236490.0    770897.0     153075.0    10834533   
2025-01-14  5990.299805  ...  299770.0    439344.0      97862.0    13330716   
2025-01-15  5876.850098  ...  709371.0    483267.0     570737.0     7011446   
2025-01-16  5916.049805  ...  306605.0    428736.0      71779.0    10606227   
2025-01-17  5916.049805  ...  360893.0    177031.0      75503.0    11556038   

Price                                                                     \
Ticker     HINDUNILVR.NS MARUTI.NS NESTLEIND.NS     ONGC.NS  RELIANCE.NS   
Date                                                                       
1996-01-01         11000       NaN          NaN     10800.0  104121369.0   
1996-01-02        203500       NaN          NaN     27000.0  168743308.0   
1996-01-03         58000       NaN          NaN     10800.0  209323879.0   
1996-01-04        111500       NaN          NaN     41400.0  216900264.0   
1996-01-05         39500       NaN          NaN     46800.0  166708467.0   
...                  ...       ...          ...         ...          ...   
2025-01-13       2595013  253524.0     804531.0  20416370.0   13764861.0   
2025-01-14       

In [7]:
ms_stockprices.columns

MultiIndex([( 'Close', 'ABBOTINDIA.NS'),
            ( 'Close', 'ASIANPAINT.NS'),
            ( 'Close', 'AUROPHARMA.NS'),
            ( 'Close', 'BAJFINANCE.NS'),
            ( 'Close', 'BALKRISIND.NS'),
            ( 'Close',       'BBTC.NS'),
            ( 'Close',   'BOSCHLTD.NS'),
            ( 'Close',  'BRITANNIA.NS'),
            ( 'Close',     'COLPAL.NS'),
            ( 'Close',   'DIVISLAB.NS'),
            ...
            ('Volume',     'COLPAL.NS'),
            ('Volume',   'DIVISLAB.NS'),
            ('Volume',  'GARFIBRES.NS'),
            ('Volume',   'HDFCBANK.NS'),
            ('Volume', 'HINDUNILVR.NS'),
            ('Volume',     'MARUTI.NS'),
            ('Volume',  'NESTLEIND.NS'),
            ('Volume',       'ONGC.NS'),
            ('Volume',   'RELIANCE.NS'),
            ('Volume',        'TCS.NS')],
           names=['Price', 'Ticker'], length=126)

In [8]:
ms_stockprices["Close"].head()

Ticker,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
Date,,,,,,,,,,,,,,,,,,
1996-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.331029,34.748363,NaN,NaN,5.462958,3.598160,NaN
1996-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.473685,NaN,NaN,NaN,2.327118,35.119434,NaN,NaN,5.546906,3.570179,NaN
1996-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.379674,NaN,NaN,NaN,2.334940,34.911400,NaN,NaN,5.488788,3.597286,NaN
1996-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.379674,NaN,NaN,NaN,2.319296,34.914215,NaN,NaN,5.490404,3.564058,NaN
1996-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.384377,NaN,NaN,NaN,2.315385,34.857998,NaN,NaN,5.492018,3.539575,NaN


In [26]:
ms_stockprices.loc[:,('Close',['ASIANPAINT.NS','BBTC.NS'])]

Price              Close             
Ticker     ASIANPAINT.NS      BBTC.NS
Date                                 
1996-01-01           NaN          NaN
1996-01-02           NaN          NaN
1996-01-03           NaN          NaN
1996-01-04           NaN          NaN
1996-01-05           NaN          NaN
...                  ...          ...
2025-01-13   2252.649902  1947.900024
2025-01-14   2239.949951  1977.300049
2025-01-15   2229.699951  1991.599976
2025-01-16   2215.600098  2000.900024
2025-01-17   2261.000000  2003.300049

[7297 rows x 2 columns]

In [9]:
companyNames = 'HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS AUROPHARMA.NS BBTC.NS BALKRISIND.NS GARFIBRES.NS COLPAL.NS NESTLEIND.NS MARUTI.NS BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS BAJFINANCE.NS'

In [10]:
companyNames 

'HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS AUROPHARMA.NS BBTC.NS BALKRISIND.NS GARFIBRES.NS COLPAL.NS NESTLEIND.NS MARUTI.NS BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS BAJFINANCE.NS'

In [11]:
companyNamesArray = companyNames.split() 

In [12]:
companyNamesArray

['HDFCBANK.NS',
 'HINDUNILVR.NS',
 'ASIANPAINT.NS',
 'RELIANCE.NS',
 'TCS.NS',
 'AUROPHARMA.NS',
 'BBTC.NS',
 'BALKRISIND.NS',
 'GARFIBRES.NS',
 'COLPAL.NS',
 'NESTLEIND.NS',
 'MARUTI.NS',
 'BRITANNIA.NS',
 'BOSCHLTD.NS',
 'ONGC.NS',
 'ABBOTINDIA.NS',
 'DIVISLAB.NS',
 'BAJFINANCE.NS']

In [13]:
len(companyNamesArray)

18

In [14]:
ms_stockprices.loc[:,('Close',companyNamesArray)]

Price             Close                                                        \
Ticker      HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS  RELIANCE.NS       TCS.NS   
Date                                                                            
1996-01-01     2.331029     34.748363           NaN     3.598160          NaN   
1996-01-02     2.327118     35.119434           NaN     3.570179          NaN   
1996-01-03     2.334940     34.911400           NaN     3.597286          NaN   
1996-01-04     2.319296     34.914215           NaN     3.564058          NaN   
1996-01-05     2.315385     34.857998           NaN     3.539575          NaN   
...                 ...           ...           ...          ...          ...   
2025-01-13  1630.849976   2451.000000   2252.649902  1239.849976  4223.769531   
2025-01-14  1646.599976   2367.949951   2239.949951  1238.750000  4166.629883   
2025-01-15  1643.050049   2373.000000   2229.699951  1252.199951  4182.920898   
2025-01-16  1652.050049   2345.000000   2215.600098  1266.449951  4140.299805   
2025-01-17  1636.750000   2354.000000   2261.000000  1302.349976  4124.299805   

Price                                                                          \
Ticker     AUROPHARMA.NS      BBTC.NS BALKRISIND.NS GARFIBRES.NS    COLPAL.NS   
Date                                                                            
1996-01-01           NaN          NaN           NaN          NaN          NaN   
1996-01-02           NaN          NaN           NaN          NaN          NaN   
1996-01-03           NaN          NaN           NaN          NaN          NaN   
1996-01-04           NaN          NaN           NaN          NaN          NaN   
1996-01-05           NaN          NaN           NaN          NaN          NaN   
...                  ...          ...           ...          ...          ...   
2025-01-13   1161.400024  1947.900024   2595.750000   754.549988  2752.050049   
2025-01-14   1174.500000  1977.300049   2705.350098   763.799988  2713.300049   
2025-01-15   1147.550049  1991.599976   2680.300049   781.549988  2664.750000   
2025-01-16   1156.300049  2000.900024   2725.300049   798.599976  2652.550049   
2025-01-17   1178.349976  2003.300049   2705.850098   812.750000  2683.649902   

Price                                                                         \
Ticker     NESTLEIND.NS     MARUTI.NS BRITANNIA.NS   BOSCHLTD.NS     ONGC.NS   
Date                                                                           
1996-01-01          NaN           NaN          NaN           NaN    5.462958   
1996-01-02          NaN           NaN     7.473685           NaN    5.546906   
1996-01-03          NaN           NaN     7.379674           NaN    5.488788   
1996-01-04          NaN           NaN     7.379674           NaN    5.490404   
1996-01-05          NaN           NaN     7.384377           NaN    5.492018   
...                 ...           ...          ...           ...         ...   
2025-01-13  2237.350098  11497.700195  4927.049805  30979.750000  255.729996   
2025-01-14  2219.050049  11741.150391  4862.399902  31621.699219  260.369995   
2025-01-15  2199.649902  11947.400391  4877.000000  31463.949219  258.179993   
2025-01-16  2169.300049  12092.299805  4844.549805  31354.349609  263.179993   
2025-01-17  2217.199951  12136.349609  4854.399902  31399.800781  266.570007   

Price                                                
Ticker     ABBOTINDIA.NS  DIVISLAB.NS BAJFINANCE.NS  
Date                                                 
1996-01-01           NaN          NaN           NaN  
1996-01-02           NaN          NaN           NaN  
1996-01-03           NaN          NaN           NaN  
1996-01-04           NaN          NaN           NaN  
1996-01-05           NaN          NaN           NaN  
...                  ...          ...           ...  
2025-01-13  28555.800781  5732.500000   7150.950195  
2025-01-14  28235.550781  5990.299805   7335.000000  
2025-01-15  277

In [15]:
stockPrices_18 = ms_stockprices.loc[:,('Close',companyNamesArray)]

In [91]:
stockPrices_18

Price             Close                                                        \
Ticker      HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS  RELIANCE.NS       TCS.NS   
Date                                                                            
1996-01-01     2.331029     34.748363           NaN     3.598160          NaN   
1996-01-02     2.327118     35.119434           NaN     3.570179          NaN   
1996-01-03     2.334940     34.911400           NaN     3.597286          NaN   
1996-01-04     2.319296     34.914215           NaN     3.564058          NaN   
1996-01-05     2.315385     34.857998           NaN     3.539575          NaN   
...                 ...           ...           ...          ...          ...   
2025-01-13  1630.849976   2451.000000   2252.649902  1239.849976  4223.769531   
2025-01-14  1646.599976   2367.949951   2239.949951  1238.750000  4166.629883   
2025-01-15  1643.050049   2373.000000   2229.699951  1252.199951  4182.920898   
2025-01-16  1652.050049   2345.000000   2215.600098  1266.449951  4140.299805   
2025-01-17  1636.750000   2354.000000   2261.000000  1302.349976  4124.299805   

Price                                                                          \
Ticker     AUROPHARMA.NS      BBTC.NS BALKRISIND.NS GARFIBRES.NS    COLPAL.NS   
Date                                                                            
1996-01-01           NaN          NaN           NaN          NaN          NaN   
1996-01-02           NaN          NaN           NaN          NaN          NaN   
1996-01-03           NaN          NaN           NaN          NaN          NaN   
1996-01-04           NaN          NaN           NaN          NaN          NaN   
1996-01-05           NaN          NaN           NaN          NaN          NaN   
...                  ...          ...           ...          ...          ...   
2025-01-13   1161.400024  1947.900024   2595.750000   754.549988  2752.050049   
2025-01-14   1174.500000  1977.300049   2705.350098   763.799988  2713.300049   
2025-01-15   1147.550049  1991.599976   2680.300049   781.549988  2664.750000   
2025-01-16   1156.300049  2000.900024   2725.300049   798.599976  2652.550049   
2025-01-17   1178.349976  2003.300049   2705.850098   812.750000  2683.649902   

Price                                                                         \
Ticker     NESTLEIND.NS     MARUTI.NS BRITANNIA.NS   BOSCHLTD.NS     ONGC.NS   
Date                                                                           
1996-01-01          NaN           NaN          NaN           NaN    5.462958   
1996-01-02          NaN           NaN     7.473685           NaN    5.546906   
1996-01-03          NaN           NaN     7.379674           NaN    5.488788   
1996-01-04          NaN           NaN     7.379674           NaN    5.490404   
1996-01-05          NaN           NaN     7.384377           NaN    5.492018   
...                 ...           ...          ...           ...         ...   
2025-01-13  2237.350098  11497.700195  4927.049805  30979.750000  255.729996   
2025-01-14  2219.050049  11741.150391  4862.399902  31621.699219  260.369995   
2025-01-15  2199.649902  11947.400391  4877.000000  31463.949219  258.179993   
2025-01-16  2169.300049  12092.299805  4844.549805  31354.349609  263.179993   
2025-01-17  2217.199951  12136.349609  4854.399902  31399.800781  266.570007   

Price                                                
Ticker     ABBOTINDIA.NS  DIVISLAB.NS BAJFINANCE.NS  
Date                                                 
1996-01-01           NaN          NaN           NaN  
1996-01-02           NaN          NaN           NaN  
1996-01-03           NaN          NaN           NaN  
1996-01-04           NaN          NaN           NaN  
1996-01-05           NaN          NaN           NaN  
...                  ...          ...           ...  
2025-01-13  28555.800781  5732.500000   7150.950195  
2025-01-14  28235.550781  5990.299805   7335.000000  
2025-01-15  277

In [92]:
stockPrices_18_array = stockPrices_18.to_numpy()

In [93]:
stockPrices_18_array

array([[2.33102942e+00, 3.47483635e+01,            nan, ...,
                   nan,            nan,            nan],
       [2.32711792e+00, 3.51194344e+01,            nan, ...,
                   nan,            nan,            nan],
       [2.33494043e+00, 3.49113998e+01,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [1.64305005e+03, 2.37300000e+03, 2.22969995e+03, ...,
        2.77456992e+04, 5.87685010e+03, 7.17739990e+03],
       [1.65205005e+03, 2.34500000e+03, 2.21560010e+03, ...,
        2.72980996e+04, 5.91604980e+03, 7.25975000e+03],
       [1.63675000e+03, 2.35400000e+03, 2.26100000e+03, ...,
        2.79296992e+04, 5.91604980e+03, 7.18210010e+03]])

In [95]:
df_afternumpy = pd.DataFrame(stockPrices_18_array)

In [96]:
df_afternumpy

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2.331029,34.748363,NaN,3.598160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.462958,NaN,NaN,NaN
1,2.327118,35.119434,NaN,3.570179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.473685,NaN,5.546906,NaN,NaN,NaN
2,2.334940,34.911400,NaN,3.597286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.379674,NaN,5.488788,NaN,NaN,NaN
3,2.319296,34.914215,NaN,3.564058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.379674,NaN,5.490404,NaN,NaN,NaN
4,2.315385,34.857998,NaN,3.539575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.384377,NaN,5.492018,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7292,1630.849976,2451.000000,2252.649902,1239.849976,4223.769531,1161.400024,1947.900024,2595.750000,754.549988,2752.050049,2237.350098,11497.700195,4927.049805,30979.750000,255.729996,28555.800781,5732.500000,7150.950195
7293,1646.599976,2367.949951,2239.949951,1238.750000,4166.629883,1174.500000,1977.300049,2705.350098,763.799988,2713.300049,2219.050049,11741.150391,4862.399902,31621.699219,260.369995,28235.550781,5990.299805,7335.000000
7294,1643.050049,2373.000000,2229.699951,1252.199951,4182.920898,1147.550049,1991.599976,2680.300049,781.549988,2664.750000,2199.649902,11947.400391,4877.000000,31463.949219,258.179993,27745.699219,5876.850098,7177.399902
7295,1652.050049,2345.000000,2215.600098,1266.449951,4140.299805,1156.300049,2000.900024,2725.300049,798.599976,2652.550049,2169.300049,12092.299805,4844.549805,31354.349609,263.179993,27298.099609,5916.049805,7259.750000


In [97]:
stockPrices_18_arrayValues = stockPrices_18.values

In [98]:
stockPrices_18_arrayValues

array([[2.33102942e+00, 3.47483635e+01,            nan, ...,
                   nan,            nan,            nan],
       [2.32711792e+00, 3.51194344e+01,            nan, ...,
                   nan,            nan,            nan],
       [2.33494043e+00, 3.49113998e+01,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [1.64305005e+03, 2.37300000e+03, 2.22969995e+03, ...,
        2.77456992e+04, 5.87685010e+03, 7.17739990e+03],
       [1.65205005e+03, 2.34500000e+03, 2.21560010e+03, ...,
        2.72980996e+04, 5.91604980e+03, 7.25975000e+03],
       [1.63675000e+03, 2.35400000e+03, 2.26100000e+03, ...,
        2.79296992e+04, 5.91604980e+03, 7.18210010e+03]])

In [57]:
new_df = pd.DataFrame(stockPrices_18.values, columns=stockPrices_18.columns)

In [58]:
new_df

Price         Close                                                        \
Ticker  HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS  RELIANCE.NS       TCS.NS   
0          2.331029     34.748363           NaN     3.598160          NaN   
1          2.327118     35.119434           NaN     3.570179          NaN   
2          2.334940     34.911400           NaN     3.597286          NaN   
3          2.319296     34.914215           NaN     3.564058          NaN   
4          2.315385     34.857998           NaN     3.539575          NaN   
...             ...           ...           ...          ...          ...   
7292    1630.849976   2451.000000   2252.649902  1239.849976  4223.769531   
7293    1646.599976   2367.949951   2239.949951  1238.750000  4166.629883   
7294    1643.050049   2373.000000   2229.699951  1252.199951  4182.920898   
7295    1652.050049   2345.000000   2215.600098  1266.449951  4140.299805   
7296    1636.750000   2354.000000   2261.000000  1302.349976  4124.299805   

Price                                                                      \
Ticker AUROPHARMA.NS      BBTC.NS BALKRISIND.NS GARFIBRES.NS    COLPAL.NS   
0                NaN          NaN           NaN          NaN          NaN   
1                NaN          NaN           NaN          NaN          NaN   
2                NaN          NaN           NaN          NaN          NaN   
3                NaN          NaN           NaN          NaN          NaN   
4                NaN          NaN           NaN          NaN          NaN   
...              ...          ...           ...          ...          ...   
7292     1161.400024  1947.900024   2595.750000   754.549988  2752.050049   
7293     1174.500000  1977.300049   2705.350098   763.799988  2713.300049   
7294     1147.550049  1991.599976   2680.300049   781.549988  2664.750000   
7295     1156.300049  2000.900024   2725.300049   798.599976  2652.550049   
7296     1178.349976  2003.300049   2705.850098   812.750000  2683.649902   

Price                                                                     \
Ticker NESTLEIND.NS     MARUTI.NS BRITANNIA.NS   BOSCHLTD.NS     ONGC.NS   
0               NaN           NaN          NaN           NaN    5.462958   
1               NaN           NaN     7.473685           NaN    5.546906   
2               NaN           NaN     7.379674           NaN    5.488788   
3               NaN           NaN     7.379674           NaN    5.490404   
4               NaN           NaN     7.384377           NaN    5.492018   
...             ...           ...          ...           ...         ...   
7292    2237.350098  11497.700195  4927.049805  30979.750000  255.729996   
7293    2219.050049  11741.150391  4862.399902  31621.699219  260.369995   
7294    2199.649902  11947.400391  4877.000000  31463.949219  258.179993   
7295    2169.300049  12092.299805  4844.549805  31354.349609  263.179993   
7296    2217.199951  12136.349609  4854.399902  31399.800781  266.570007   

Price                                            
Ticker ABBOTINDIA.NS  DIVISLAB.NS BAJFINANCE.NS  
0                NaN          NaN           NaN  
1                NaN          NaN           NaN  
2                NaN          NaN           NaN  
3                NaN          NaN           NaN  
4                NaN          NaN           NaN  
...              ...          ...           ...  
7292    28555.800781  5732.500000   7150.950195  
7293    28235.550781  5990.299805   7335.000000  
7294    27745.699219  5876.850098   7177.399902  
7295    27298.099609  5916.049805   7259.750000  
7296    27929.699219  5916.049805   7182.100098  

[7297 rows x 18 columns]

In [59]:
print(new_df.columns)

MultiIndex([('Close',   'HDFCBANK.NS'),
            ('Close', 'HINDUNILVR.NS'),
            ('Close', 'ASIANPAINT.NS'),
            ('Close',   'RELIANCE.NS'),
            ('Close',        'TCS.NS'),
            ('Close', 'AUROPHARMA.NS'),
            ('Close',       'BBTC.NS'),
            ('Close', 'BALKRISIND.NS'),
            ('Close',  'GARFIBRES.NS'),
            ('Close',     'COLPAL.NS'),
            ('Close',  'NESTLEIND.NS'),
            ('Close',     'MARUTI.NS'),
            ('Close',  'BRITANNIA.NS'),
            ('Close',   'BOSCHLTD.NS'),
            ('Close',       'ONGC.NS'),
            ('Close', 'ABBOTINDIA.NS'),
            ('Close',   'DIVISLAB.NS'),
            ('Close', 'BAJFINANCE.NS')],
           names=['Price', 'Ticker'])


In [82]:
singleIndex_df = new_df.groupby(level=1,axis=1).sum()

/tmp/ipykernel_385/3502407195.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  singleIndex_df = new_df.groupby(level=1,axis=1).sum()


In [84]:
singleIndex_df

Ticker,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.331029,34.748363,0.000000,0.000000,5.462958,3.598160,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.473685,0.000000,0.000000,0.000000,2.327118,35.119434,0.000000,0.000000,5.546906,3.570179,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.334940,34.911400,0.000000,0.000000,5.488788,3.597286,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.319296,34.914215,0.000000,0.000000,5.490404,3.564058,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.384377,0.000000,0.000000,0.000000,2.315385,34.857998,0.000000,0.000000,5.492018,3.539575,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7292,28555.800781,2252.649902,1161.400024,7150.950195,2595.750000,1947.900024,30979.750000,4927.049805,2752.050049,5732.500000,754.549988,1630.849976,2451.000000,11497.700195,2237.350098,255.729996,1239.849976,4223.769531
7293,28235.550781,2239.949951,1174.500000,7335.000000,2705.350098,1977.300049,31621.699219,4862.399902,2713.300049,5990.299805,763.799988,1646.599976,2367.949951,11741.150391,2219.050049,260.369995,1238.750000,4166.629883
7294,27745.699219,2229.699951,1147.550049,7177.399902,2680.300049,1991.599976,31463.949219,4877.000000,2664.750000,5876.850098,781.549988,1643.050049,2373.000000,11947.400391,2199.649902,258.179993,1252.199951,4182.920898
7295,27298.099609,2215.600098,1156.300049,7259.750000,2725.300049,2000.900024,31354.349609,4844.549805,2652.550049,5916.049805,798.599976,1652.050049,2345.000000,12092.299805,2169.300049,263.179993,1266.449951,4140.299805


In [86]:
singleIndex_df.index

RangeIndex(start=0, stop=7297, step=1)

In [87]:
singleIndex_df_withoutindex = singleIndex_df.reset_index(drop=True)

In [88]:
singleIndex_df_withoutindex

Ticker,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.331029,34.748363,0.000000,0.000000,5.462958,3.598160,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.473685,0.000000,0.000000,0.000000,2.327118,35.119434,0.000000,0.000000,5.546906,3.570179,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.334940,34.911400,0.000000,0.000000,5.488788,3.597286,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.319296,34.914215,0.000000,0.000000,5.490404,3.564058,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.384377,0.000000,0.000000,0.000000,2.315385,34.857998,0.000000,0.000000,5.492018,3.539575,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7292,28555.800781,2252.649902,1161.400024,7150.950195,2595.750000,1947.900024,30979.750000,4927.049805,2752.050049,5732.500000,754.549988,1630.849976,2451.000000,11497.700195,2237.350098,255.729996,1239.849976,4223.769531
7293,28235.550781,2239.949951,1174.500000,7335.000000,2705.350098,1977.300049,31621.699219,4862.399902,2713.300049,5990.299805,763.799988,1646.599976,2367.949951,11741.150391,2219.050049,260.369995,1238.750000,4166.629883
7294,27745.699219,2229.699951,1147.550049,7177.399902,2680.300049,1991.599976,31463.949219,4877.000000,2664.750000,5876.850098,781.549988,1643.050049,2373.000000,11947.400391,2199.649902,258.179993,1252.199951,4182.920898
7295,27298.099609,2215.600098,1156.300049,7259.750000,2725.300049,2000.900024,31354.349609,4844.549805,2652.550049,5916.049805,798.599976,1652.050049,2345.000000,12092.299805,2169.300049,263.179993,1266.449951,4140.299805


In [90]:
print(singleIndex_df_withoutindex.columns)

Index(['ABBOTINDIA.NS', 'ASIANPAINT.NS', 'AUROPHARMA.NS', 'BAJFINANCE.NS',
       'BALKRISIND.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'COLPAL.NS',
       'DIVISLAB.NS', 'GARFIBRES.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS',
       'MARUTI.NS', 'NESTLEIND.NS', 'ONGC.NS', 'RELIANCE.NS', 'TCS.NS'],
      dtype='object', name='Ticker')


In [79]:
singleIndex_df["Date"] = stockPrices_18.index

In [80]:
singleIndex_df.set_index("Date", inplace=True)

In [81]:
singleIndex_df

Ticker,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
Date,,,,,,,,,,,,,,,,,,
1996-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.331029,34.748363,0.000000,0.000000,5.462958,3.598160,0.000000
1996-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.473685,0.000000,0.000000,0.000000,2.327118,35.119434,0.000000,0.000000,5.546906,3.570179,0.000000
1996-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.334940,34.911400,0.000000,0.000000,5.488788,3.597286,0.000000
1996-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.319296,34.914215,0.000000,0.000000,5.490404,3.564058,0.000000
1996-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.384377,0.000000,0.000000,0.000000,2.315385,34.857998,0.000000,0.000000,5.492018,3.539575,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-13,28555.800781,2252.649902,1161.400024,7150.950195,2595.750000,1947.900024,30979.750000,4927.049805,2752.050049,5732.500000,754.549988,1630.849976,2451.000000,11497.700195,2237.350098,255.729996,1239.849976,4223.769531
2025-01-14,28235.550781,2239.949951,1174.500000,7335.000000,2705.350098,1977.300049,31621.699219,4862.399902,2713.300049,5990.299805,763.799988,1646.599976,2367.949951,11741.150391,2219.050049,260.369995,1238.750000,4166.629883
2025-01-15,27745.699219,2229.699951,1147.550049,7177.399902,2680.300049,1991.599976,31463.949219,4877.000000,2664.750000,5876.850098,781.549988,1643.050049,2373.000000,11947.400391,2199.649902,258.179993,1252.199951,4182.920898


In [67]:
stockPrices_18.index

DatetimeIndex(['1996-01-01', '1996-01-02', '1996-01-03', '1996-01-04',
               '1996-01-05', '1996-01-08', '1996-01-09', '1996-01-10',
               '1996-01-11', '1996-01-12',
               ...
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-13', '2025-01-14', '2025-01-15',
               '2025-01-16', '2025-01-17'],
              dtype='datetime64[ns]', name='Date', length=7297, freq=None)

In [65]:
singleIndex_df_2 = stockPrices_18.groupby(level=1,axis=1).sum()

/tmp/ipykernel_385/1272841217.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  singleIndex_df_2 = stockPrices_18.groupby(level=1,axis=1).sum()


In [101]:
singleIndex_df_2

Ticker,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
Date,,,,,,,,,,,,,,,,,,
1996-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.331029,34.748363,0.000000,0.000000,5.462958,3.598160,0.000000
1996-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.473685,0.000000,0.000000,0.000000,2.327118,35.119434,0.000000,0.000000,5.546906,3.570179,0.000000
1996-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.334940,34.911400,0.000000,0.000000,5.488788,3.597286,0.000000
1996-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.319296,34.914215,0.000000,0.000000,5.490404,3.564058,0.000000
1996-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.384377,0.000000,0.000000,0.000000,2.315385,34.857998,0.000000,0.000000,5.492018,3.539575,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-13,28555.800781,2252.649902,1161.400024,7150.950195,2595.750000,1947.900024,30979.750000,4927.049805,2752.050049,5732.500000,754.549988,1630.849976,2451.000000,11497.700195,2237.350098,255.729996,1239.849976,4223.769531
2025-01-14,28235.550781,2239.949951,1174.500000,7335.000000,2705.350098,1977.300049,31621.699219,4862.399902,2713.300049,5990.299805,763.799988,1646.599976,2367.949951,11741.150391,2219.050049,260.369995,1238.750000,4166.629883
2025-01-15,27745.699219,2229.699951,1147.550049,7177.399902,2680.300049,1991.599976,31463.949219,4877.000000,2664.750000,5876.850098,781.549988,1643.050049,2373.000000,11947.400391,2199.649902,258.179993,1252.199951,4182.920898


In [100]:
print(singleIndex_df_2.columns)

Index(['ABBOTINDIA.NS', 'ASIANPAINT.NS', 'AUROPHARMA.NS', 'BAJFINANCE.NS',
       'BALKRISIND.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'COLPAL.NS',
       'DIVISLAB.NS', 'GARFIBRES.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS',
       'MARUTI.NS', 'NESTLEIND.NS', 'ONGC.NS', 'RELIANCE.NS', 'TCS.NS'],
      dtype='object', name='Ticker')


In [103]:
singleIndex_df_2.index

DatetimeIndex(['1996-01-01', '1996-01-02', '1996-01-03', '1996-01-04',
               '1996-01-05', '1996-01-08', '1996-01-09', '1996-01-10',
               '1996-01-11', '1996-01-12',
               ...
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-13', '2025-01-14', '2025-01-15',
               '2025-01-16', '2025-01-17'],
              dtype='datetime64[ns]', name='Date', length=7297, freq=None)

In [104]:
singleIndex_df_2= singleIndex_df_2.reset_index()

In [105]:
singleIndex_df_2

Ticker,Date,ABBOTINDIA.NS,ASIANPAINT.NS,AUROPHARMA.NS,BAJFINANCE.NS,BALKRISIND.NS,BBTC.NS,BOSCHLTD.NS,BRITANNIA.NS,COLPAL.NS,DIVISLAB.NS,GARFIBRES.NS,HDFCBANK.NS,HINDUNILVR.NS,MARUTI.NS,NESTLEIND.NS,ONGC.NS,RELIANCE.NS,TCS.NS
0,1996-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.331029,34.748363,0.000000,0.000000,5.462958,3.598160,0.000000
1,1996-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.473685,0.000000,0.000000,0.000000,2.327118,35.119434,0.000000,0.000000,5.546906,3.570179,0.000000
2,1996-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.334940,34.911400,0.000000,0.000000,5.488788,3.597286,0.000000
3,1996-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.379674,0.000000,0.000000,0.000000,2.319296,34.914215,0.000000,0.000000,5.490404,3.564058,0.000000
4,1996-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.384377,0.000000,0.000000,0.000000,2.315385,34.857998,0.000000,0.000000,5.492018,3.539575,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7292,2025-01-13,28555.800781,2252.649902,1161.400024,7150.950195,2595.750000,1947.900024,30979.750000,4927.049805,2752.050049,5732.500000,754.549988,1630.849976,2451.000000,11497.700195,2237.350098,255.729996,1239.849976,4223.769531
7293,2025-01-14,28235.550781,2239.949951,1174.500000,7335.000000,2705.350098,1977.300049,31621.699219,4862.399902,2713.300049,5990.299805,763.799988,1646.599976,2367.949951,11741.150391,2219.050049,260.369995,1238.750000,4166.629883
7294,2025-01-15,27745.699219,2229.699951,1147.550049,7177.399902,2680.300049,1991.599976,31463.949219,4877.000000,2664.750000,5876.850098,781.549988,1643.050049,2373.000000,11947.400391,2199.649902,258.179993,1252.199951,4182.920898
7295,2025-01-16,27298.099609,2215.600098,1156.300049,7259.750000,2725.300049,2000.900024,31354.349609,4844.549805,2652.550049,5916.049805,798.599976,1652.050049,2345.000000,12092.299805,2169.300049,263.179993,1266.449951,4140.299805


In [106]:
# Melt the DataFrame
df_long = singleIndex_df_2.melt(id_vars=["Date"], var_name='company', value_name='price')

In [107]:
df_long

,Date,company,price
0,1996-01-01,ABBOTINDIA.NS,0.000000
1,1996-01-02,ABBOTINDIA.NS,0.000000
2,1996-01-03,ABBOTINDIA.NS,0.000000
3,1996-01-04,ABBOTINDIA.NS,0.000000
4,1996-01-05,ABBOTINDIA.NS,0.000000
...,...,...,...
131341,2025-01-13,TCS.NS,4223.769531
131342,2025-01-14,TCS.NS,4166.629883
131343,2025-01-15,TCS.NS,4182.920898
131344,2025-01-16,TCS.NS,4140.299805
